In [ ]:
%pip install axelrod

Se creará un juego con 4 estrategias, correspondiente al "juego de la corrupción"

In [7]:
import axelrod as axl

# Definir estrategias
strategies = [axl.TitForTat(), axl.Random(), axl.Defector(), axl.Cooperator()]

# Crear el torneo
tournament = axl.Tournament(players=strategies)

# Ejecutar el torneo
results = tournament.play()

# Mostrar los resultados del torneo
print("Resultados del torneo:")
for player in results.ranked_names:
    print(player)
    
# Obtener la matriz de pagos
payoff_matrix = results.payoff_matrix
print("Matriz de pagos:")
print(payoff_matrix)

# Obtener la puntuación promedio de cada jugador
average_scores = results.summarise()
print("Puntuación promedio de cada jugador:")
print(average_scores)



Analysing: 100%|██████████| 25/25 [00:00<00:00, 108.76it/s]

Resultados del torneo:
Defector
Random: 0.5
Tit For Tat
Cooperator
Matriz de pagos:
[[3.0, 2.246, 0.9949999999999999, 3.0], [2.256, 2.2662500000000003, 0.49450000000000005, 3.9799999999999995], [1.02, 3.022, 1.0, 5.0], [3.0, 1.53, 0.0, 3.0]]
Puntuación promedio de cada jugador:
[Player(Rank=0, Name='Defector', Median_score=3.006666666666667, Cooperation_rating=0.0, Wins=3.0, Initial_C_rate=0.0, CC_rate=0.0, CD_rate=0.0, DC_rate=0.5035, DD_rate=0.4965, CC_to_C_rate=0, CD_to_C_rate=0, DC_to_C_rate=0, DD_to_C_rate=0), Player(Rank=1, Name='Random: 0.5', Median_score=2.2399999999999998, Cooperation_rating=0.5051666666666667, Wins=1.0, Initial_C_rate=0.4, CC_rate=0.25400000000000006, CD_rate=0.2511666666666667, DC_rate=0.2466666666666667, DD_rate=0.24816666666666667, CC_to_C_rate=0.5117811713850406, CD_to_C_rate=0.5153791441823896, DC_to_C_rate=0.5003682763842254, DD_to_C_rate=0.4907110748352358), Player(Rank=2, Name='Tit For Tat', Median_score=2.0808333333333335, Cooperation_rating=0.502333

Se crea la matriz del juego de la corrupción

In [8]:
pip install axelrod numpy matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
import axelrod as axl
from typing import List, Union
import random
import numpy as np
import matplotlib.pyplot as plt

# Definición de la matriz de pagos global
PAYOFF_MATRIX = {
    'C': {'C': 1, 'N': -2, 'PC': 1, 'PN': -2},
    'N': {'C': 2, 'N': 0, 'PC': 1, 'PN': -1},
    'PC': {'C': 1, 'N': -3, 'PC': 1, 'PN': -4},
    'PN': {'C': 2, 'N': -1, 'PC': 0, 'PN': -4}
}

class ModifiedTitForTat(axl.Player):
    """
    Una versión modificada de Tit For Tat para la matriz expandida.
    Comienza cooperando y luego imita la última acción del oponente.
    """
    name = 'Modified Tit For Tat'
    
    def __init__(self) -> None:
        super().__init__()
        self.history = []
        self.actions = ['C', 'N', 'PC', 'PN']
    
    def strategy(self, opponent: axl.Player) -> str:
        """Implementa la estrategia para 4 acciones posibles"""
        if len(opponent.history) == 0:
            return 'C'  # Comienza cooperando
        
        # Imita la última acción del oponente
        return opponent.history[-1]

class GradualStrategy(axl.Player):
    """
    Una estrategia que responde gradualmente a las defecciones.
    - Si el oponente coopera, coopera
    - Si defecta, incrementa gradualmente el nivel de no cooperación
    """
    name = 'Gradual Strategy'
    
    def __init__(self) -> None:
        super().__init__()
        self.history = []
        self.defection_count = 0
        self.actions = ['C', 'N', 'PC', 'PN']
    
    def strategy(self, opponent: axl.Player) -> str:
        if len(opponent.history) == 0:
            return 'C'
            
        if opponent.history[-1] == 'C':
            self.defection_count = max(0, self.defection_count - 1)
        else:
            self.defection_count += 1
            
        # Escala la respuesta basada en el número de defecciones
        if self.defection_count == 0:
            return 'C'
        elif self.defection_count == 1:
            return 'N'
        elif self.defection_count == 2:
            return 'PC'
        else:
            return 'PN'

class AdaptiveStrategy(axl.Player):
    name = 'Adaptive Strategy'
    
    def __init__(self) -> None:
        super().__init__()
        self.actions = ['C', 'N', 'PC', 'PN']
        self.scores = {'C': 0, 'N': 0, 'PC': 0, 'PN': 0}
    
    def strategy(self, opponent: axl.Player) -> str:
        if not opponent.history:  # Usar la propiedad history en lugar de len()
            return 'C'
            
        # Actualiza los puntajes basado en la última interacción
        if self.history:  # Usar la propiedad history
            self.update_scores(self.history[-1], opponent.history[-1])
            
        best_action = max(self.scores.items(), key=lambda x: x[1])[0]
        return best_action

class RandomStrategy(axl.Player):
    """
    Una estrategia que elige acciones al azar
    """
    name = 'Random Strategy'
    
    def __init__(self) -> None:
        super().__init__()
        self.history = []
        self.actions = ['C', 'N', 'PC', 'PN']
    
    def strategy(self, opponent: axl.Player) -> str:
        return random.choice(self.actions)

def play_match(player1: axl.Player, player2: axl.Player, rounds: int = 100) -> List[tuple]:
    """
    Juega un match entre dos jugadores usando la matriz de pagos expandida
    """
    history = []
    for _ in range(rounds):
        action1 = player1.strategy(player2)
        action2 = player2.strategy(player1)
        
        score1 = PAYOFF_MATRIX[action1][action2]
        score2 = PAYOFF_MATRIX[action2][action1]
        
        history.append((action1, action2, score1, score2))
        
        player1.history.append(action1)
        player2.history.append(action2)
        
    return history

def analyze_match(history: List[tuple], player1_name: str, player2_name: str) -> None:
    """
    Analiza y muestra los resultados de un match
    """
    player1_scores = [h[2] for h in history]
    player2_scores = [h[3] for h in history]
    
    print(f"\nResultados del match entre {player1_name} vs {player2_name}:")
    print(f"Puntaje total {player1_name}: {sum(player1_scores)}")
    print(f"Puntaje total {player2_name}: {sum(player2_scores)}")
    
    # Graficar la evolución de los puntajes
    plt.figure(figsize=(10, 6))
    plt.plot(np.cumsum(player1_scores), label=player1_name)
    plt.plot(np.cumsum(player2_scores), label=player2_name)
    plt.xlabel('Ronda')
    plt.ylabel('Puntaje acumulado')
    plt.title(f'Evolución de puntajes: {player1_name} vs {player2_name}')
    plt.legend()
    plt.grid(True)
    plt.show()

def run_tournament(rounds: int = 100) -> None:
    """
    Ejecuta un torneo entre todas las estrategias
    """
    strategies = [
        ModifiedTitForTat(),
        GradualStrategy(),
        AdaptiveStrategy(),
        RandomStrategy()
    ]
    
    results = {}
    
    for i, player1 in enumerate(strategies):
        for j, player2 in enumerate(strategies[i:], i):
            if i != j:
                history = play_match(player1, player2, rounds)
                analyze_match(history, player1.name, player2.name)
                
                total_score1 = sum(h[2] for h in history)
                total_score2 = sum(h[3] for h in history)
                
                results[(player1.name, player2.name)] = (total_score1, total_score2)
    
    return results

if __name__ == "__main__":
    # Ejecutar un torneo con 100 rondas
    print("Iniciando torneo...")
    tournament_results = run_tournament(100)
    
    # Mostrar resultados finales
    print("\nResultados finales del torneo:")
    for match, scores in tournament_results.items():
        print(f"{match[0]} vs {match[1]}: {scores[0]} - {scores[1]}")

Iniciando torneo...


AttributeError: property 'history' of 'ModifiedTitForTat' object has no setter